# General API quickstart

In [ ]:
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import theano.tensor as tt

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
%config InlineBackend.figure_format='retina'
az.style.use('arviz-darkgrid')
print(f'Running on PyMC3 v{pm.__version__}')
print(f'Running on ArviZ v{az.__version__}')

## Model creation

Models in PyMC3 are centered around the `Model` class. It has references to all random variables (RVs) and computes the model logp and its gradients. Usually, you would instantiate it as part of a `with` context:

In [ ]:
with pm.Model() as model:
    # Model definition
    pass

We discuss RVs further below but let's create a simple model to explore the `Model` class.

In [ ]:
with pm.Model() as model:
    mu = pm.Normal('mu', mu=0, sigma=1)
    obs = pm.Normal('obs', mu=mu, sigma=1, observed=np.random.randn(100))

In [ ]:
model.basic_RVs

In [ ]:
model.free_RVs

In [ ]:
model.observed_RVs

In [ ]:
model.logp({'mu': 0})

It's worth highlighting the design choice we made with logp. As you can see above, `logp` is being called with arguments, so it's a method of the model instance.  More precisely, it puts together a function based on the current state of the model - or on the state given as an argument to `logp` (see example below).

For diverse reasons, we assume a `Model` instance isn't static. If you need to use `logp` in an inner loop and it needs to be static, simply use something like `logp = model.logp`. Here is an example below - note the caching effect and the speedup.

In [ ]:
%timeit model.logp({mu: 0.1})
logp = model.logp
%timeit logp({mu: 0.1})

## Probability distributions

Every probabilistic program consists of observed and unobserved Random Variables (RVs). Observed RVs are defined via likelihood distributions, while unobserved RVs are defined via prior distributions. In PyMC3, probability distributions are available from the main module space.

In [ ]:
help(pm.Normal)

In [ ]:
dir(pm.distributions.mixture)

### Unobserved Random Variables

Every unobserved RV has the following calling signature: name(str), parameter keyword arguments. Thus, a normal prior can be defined in a model context like this:

In [ ]:
with pm.Model():
    x = pm.Normal('x', mu=0, sigma=1)

As with the model, we can evaluate its logp:

In [ ]:
x.logp({'x': 0})

### Observed Random Variables

Observed RVs are defined just like unobserved RVs but require data to be passed into the observed keyword argument:

In [ ]:
with pm.Model():
    obs = pm.Normal('obs', mu=0, sigma=1, observed=np.random.randn(100))

In [ ]:
obs.logp({'mu': 0})

The `observed` keyword supports values of type `list`, `numpy.ndarray`, `theano`, and `pandas` data structures.

### Deterministic transforms

PyMC3 allows you to freely do algebra in all kinds of ways:

In [ ]:
with pm.Model():
    x = pm.Normal('x', mu=0, sigma=1)
    y = pm.Gamma('y', alpha=1, beta=1)
    plus_2 = x + 2
    summed = x + y
    squared = x ** 2
    sined = pm.math.sin(x)

While these transforms work seamlessly, their results are **not** stored automatically. Thus, if you want to keep track of a transformed variable, you must use `pm.Deterministic`.

In [ ]:
with pm.Model():
    x = pm.Normal('x', mu=0, sigma=1)
    plus_2 = pm.Deterministic('plus_2', x + 2)

Note that `plus_2` can be used in identical to the above, we only tell PyMC3 to keep track of this RV for us.

### Automatic transform of bounded RVs

In order to sample models more efficiently, PyMC3 automatically transforms bounded RVs to be **unbounded**.

In [ ]:
with pm.Model() as model:
    x = pm.Uniform('x', lower=0, upper=1)

When we look at the RVs of the model, we would expect to find `x` there; however:

In [ ]:
model.free_RVs

The variable, `x_interval__`, represents `x` transformed to accept parameter values between `-inf` and `+inf`. In the case of an upper and lower bound, a `LogOdds` transform is applied. Sampling in this transformed space makes it easier for the sampler. PyMC3 also keeps track of the non-transformed bounded parameters. These are common deterministics (see above):

In [ ]:
model.deterministics

When displaying results, PyMC3 will usually hide transformed parameters. You can pass the `include_transformed=True` parameter to many functions to see the transformed parameters that are used for sampling.

You can also turn transforms off:

In [ ]:
with pm.Model() as model:
    x = pm.Uniform('x', lower=0, upper=1, transform=None)

print(model.free_RVs)

Or specify different transformations other than the default:

In [ ]:
import pymc3.distributions.transforms as tr

with pm.Model() as model:
    # Use the default log transform
    x1 = pm.Gamma('x1', alpha=1, beta=1)
    # Specify a different transformation
    x2 = pm.Gamma('x2', alpha=1, beta=1, transform=tr.log_exp_m1)

print(f'The default transformation of x1 is: {x1.transformation.name}')
print(f'The user-specified transformation of x2 is: {x2.transformation.name}')

### Transformed distributions and changes of variables

PyMC3 does **not** provide explicit functionality to transform one distribution to another. Instead, a dedicated distribution is usually created in consideration of optimising performance. However, users can still create transformed distribution by passing the inverse transformation to `transform` `kwarg`. Take the classic textbook example of `LogNormal`: $\mathcal log(y) \sim \mathrm Normal(\mu, \sigma)$.

In [ ]:
class Exp(tr.ElemwiseTransform):
    name = 'exp'

    def backward(self, x):
        return tt.log(x)

    def forward(self, x):
        return tt.exp(x)

    def jacobian_det(self, x):
        return -tt.log(x)

with pm.Model() as model:
    x1 = pm.Normal('x1', 0.0, 1.0, transform=Exp())
    x2 = pm.Lognormal('x2', 0.0, 1.0)

lognormal = model.named_vars['x1_exp__']
lognorm2 = model.named_vars['x2']

_, ax = plt.subplots(figsize=(5, 3))
x = np.linspace(0.0, 10.0, 100)
ax.plot(
    x,
    np.exp(lognormal.distribution.logp(x).eval()),
    '--',
    alpha=0.5,
    label='log(y) ~ Normal(0, 1)',
)
ax.plot(
    x,
    np.exp(lognorm2.distribution.logp(x).eval()),
    alpha=0.5,
    label='y ~ lognormal(0, 1)',
)
plt.legend()

Notice from above that the named variable `x1_exp__` in the `model` is Lognormal distributed.

Using a similar approach, we can create ordered RVs following some distribution. For example, we can combine the ordered transformation and logodds transformation using `Chain` to create a 2D RV that satisfy $\mathcal x1, x2 \sim \mathrm Uniform(0, 1) \mathcal\ and\ x1 < x2$.

In [ ]:
Order = tr.Ordered()
Logodd = tr.LogOdds()
chain_tran = tr.Chain([Logodd, Order])

with pm.Model() as m0:
    x = pm.Uniform('x', 0.0, 1.0, shape=2, transform=chain_tran, testval=[0.1, 0.9])
    trace = pm.sample(5000, tune=1000, progressbar=False, return_inferencedata=False)

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(10, 5))
for ivar, varname in enumerate(trace.varnames):
    ax[ivar].scatter(trace[varname][:, 0], trace[varname][:, 1], alpha=0.01)
    ax[ivar].set_xlabel(f'{varname}[0]')
    ax[ivar].set_ylabel(f'{varname}[1]')
    ax[ivar].set_title(varname)
plt.tight_layout()

### List of RVs / higher-dimensional RVs

Above we have seen how to create scalar RVs. In many models, you want multiple RVs. There is a tendency (mainly inherited from PyMC 2.x) to create a list of RVs, like:

In [ ]:
with pm.Model():
    # bad
    x = [pm.Normal(f'x_[{i}]', mu=0, sigma=1) for i in range(10)]

However, even though this works, it is quite slow and not recommended. Instead, use the `shape` kwarg.

In [ ]:
with pm.Model() as model:
    # good
    x = pm.Normal('x', mu=0, sigma=1, shape=10)

`x` is now a random vector of length 10. We can index into it or do linear algebra operations on it.

In [ ]:
with model:
    y = x[0] * x[1]  # full indexing is supported
    x.dot(x.T)  # linear algebra is supported

### Initialization with test values

While PyMC3 tries to automatically initialize models, it is sometimes helpful to define initial values for RVs. This can be done via the `testval` kwarg:

In [ ]:
with pm.Model():
    x = pm.Normal('x', mu=0, sigma=1, shape=5)

x.tag.test_value

In [ ]:
with pm.Model():
    x = pm.Normal('x', mu=0, sigma=1, shape=5, testval=np.random.randn(5))

x.tag.test_value

This technique is quite useful to identify problems with model specification or initialization.

## Inference

Once we have defined our model, we have to perform inference to approximate the posterior distribution. PyMC3 supports two broad classes of inference: sampling and variational inference.

### Sampling

The main entry point to MCMC sampling algorithms is via the `pm.sample()` function. By default, this function tries to auto-assign the right sampler(s) and auto-initialize if you don't pass anything.

With PyMC3 version >= 3.9, the `return_inferencedata=True` keyword argument makes the sample function return an `arviz.InferenceData` object instead of a `MultiTrace`. `InferenceData` has many advantages compared to a `MultiTrace`. For example, it can be saved to / loaded from a file, and it can also carry additional (meta)data such as date/version or posterior predictive distributions. Take a look at the [ArviZ Quickstart](https://arviz-devs.github.io/arviz/getting_started/Introduction.html) to learn more.

In [ ]:
with pm.Model() as model:
    mu = pm.Normal('mu', mu=0, sigma=1)
    obs = pm.Normal('obs', mu=mu, sigma=1, observed=np.random.randn(100))

    idata = pm.sample(2000, tune=1500, return_inferencedata=True)

As you can see, on a continuous model, PyMC3 assigns the NUTS sampler, which is very efficient even for complex models. PyMC3 also runs tuning to find good starting parameters for the sampler. Here we draw 2000 samples from the posterior in each chain and allow the sampler to adjust its parameters in an additional 1500 iterations. If not set via the `cores` keyword arg, the number of chains is determined from the number of available CPU cores.

In [ ]:
idata.posterior.dims

The tuning samples are discarded by default. With `discard_tuned_samples=False`, they can be kept and end up in a special property of the `InferenceData` object.

You can also run multiple chains in parallel using the `chains` and `cores` keyword args.

In [ ]:
with pm.Model() as model:
    mu = pm.Normal('mu', mu=0, sigma=1)
    obs = pm.Normal('obs', mu=mu, sigma=1, observed=np.random.randn(100))

    idata = pm.sample(cores=4, chains=6, return_inferencedata=True)

In [ ]:
idata.posterior['mu'].shape

In [ ]:
# Get values of a single chain
idata.posterior['mu'].sel(chain=1).shape

PyMC3 offers a variety of other samplers, found in `pm.step_methods`.

In [ ]:
list(filter(lambda n: n[0].upper(), dir(pm.step_methods)))

Commonly used methods step-methods, besides NUTS are `Metropolis` and `Slice`. **For almost all continuous models, "NUTS" should be preferred**. There are hard-to-sample models for which NUTS will be very slow causing many users to use `Metropolis` instead. This practice, however, is rarely successful. NUTS is very fast on simple models but can be slow if the model is very complex or is badly initialized. In the case of a complex model that is hard for NUTS, Metropolis, while faster, will have a very low effective sample size or not converge properly at all. A better approach is to instead try to improve initialization of NUTS, or reparameterize the model.

For completeness, other sampling methods can be passed to sample:

In [ ]:
with pm.Model() as model:
    mu = pm.Normal('mu', mu=0, sigma=1)
    obs = pm.Normal('obs', mu=mu, sigma=1, observed=np.random.randn(100))

    step = pm.Metropolis()
    trace = pm.sample(1000, step=step)

You can also assign variables to different step methods.

In [ ]:
with pm.Model() as model:
    mu = pm.Normal('mu', mu=0, sigma=1)
    sd = pm. HalfNormal('sd', sigma=1)
    obs = pm.Normal('obs', mu=mu, sigma=sd, observed=np.random.randn(100))

    step1 = pm.Metropolis(vars=[mu])
    step2 = pm.Slice(vars=[sd])
    idata = pm.sample(10000, step=[step1, step2], cores=4, return_inferencedata=True)

### Analyze sampling results

The most common used plot to analyze sampling results is the so-called trace-plot:

In [ ]:
az.plot_trace(idata)

Another common tactic is to look at R-hat, also known as the Gelman-Rubin statistic:

In [ ]:
az.summary(idata)

These are also part of the `forestplot`:

In [ ]:
az.plot_forest(idata, r_hat=True)

Finally, for a plot of the posterior that is inspired by the book, [Doing Bayesian Data Analysis](http://www.indiana.edu/~kruschke/DoingBayesianDataAnalysis/), you can use:

In [ ]:
az.plot_posterior(idata)

For high-dimensional models it becomes cumbersome to look at all the parameter's traces. When using NUTS we can look at the energy plot to assess problems of convergence:

In [ ]:
with pm.Model() as model:
    x = pm.Normal('x', mu=0, sigma=1, shape=100)
    idata = pm.sample(cores=4, return_inferencedata=True)

az.plot_energy(idata)

For more information on stats and the energy plot, see [here](https://docs.pymc.io/en/v3/pymc-examples/examples/diagnostics_and_criticism/sampler-stats.html). For more information on identifying sampling problems and what to do about them, see [here](https://docs.pymc.io/en/v3/pymc-examples/examples/diagnostics_and_criticism/Diagnosing_biased_Inference_with_Divergences.html).

### Variational inference

PyMC3 supports various Variational Inference techniques. While these methods are much faster, they are often also less accurate and can lead to biased inference. The main entry point is `pymc3.fit()`.

In [ ]:
with pm.Model() as model:
    mu = pm.Normal('mu', mu=0, sigma=1)
    sd = pm.HalfNormal('sd', sigma=1)
    obs = pm.Normal('obs', mu=mu, sigma=sd, observed=np.random.randn(100))

    approx = pm.fit()

The returned approximation object has various capabilities, like drawing samples from the approximated posterior, which we can analyze like a regular sampling run:

In [ ]:
approx.sample(500)

The variational submodule offers a lot of flexibility in which VI uses and follows an object-oriented design. For example, full rank ADVI estimates a full covariance matrix:

In [ ]:
mu = pm.floatX([0.0, 0.0])
cov = pm.floatX([[1, 0.5], [0.5, 1.0]])
with pm.Model() as model:
    pm.MvNormal('x', mu=mu, cov=cov, shape=2)
    approx = pm.fit(method='fullrank_advi')

An equivalent expression using the object-oriented interface is:

In [ ]:
with pm.Model() as model:
    pm.MvNormal('x', mu=mu, cov=cov, shape=2)
    approx = pm.FullRankADVI().fit()

In [ ]:
plt.figure()
trace = approx.sample(10000)
az.plot_kde(trace['x'][:, 0], trace['x'][:, 1])

Stein Variational Gradient Descent (SVGD) uses particles to estimate the posterior:

In [ ]:
w = pm.floatX([0.2, 0.8])
mu = pm.floatX([-0.3, 0.5])
sd = pm.floatX([0.1, 0.1])
with pm.Model() as model:
    pm.NormalMixture('x', w=w, mu=mu, sigma=sd)
    approx = pm.fit(method=pm.SVGD(n_particles=200, jitter=1.0))

In [ ]:
plt.figure()
trace = approx.sample(10000)
az.plot_dist(trace['x'])

For more information on variational inference, see [these examples](http://pymc-devs.github.io/pymc3/examples.html#variational-inference).

## Posterior Predictive Sampling

The `sample_posterior_predictive()` function performs predictions on hold-out data and posterior predictive checks.

In [ ]:
data = np.random.randn(100)
with pm.Model() as model:
    mu = pm.Normal('mu', mu=0, sigma=1)
    sd = pm.HalfNormal('sd', sigma=1)
    obs = pm.Normal('obs', mu=mu, sigma=sd, observed=data)

    idata = pm.sample(return_inferencedata=True)

In [ ]:
with model:
    post_pred = pm.sample_posterior_predictive(idata.posterior)

az.concat(idata, az.from_pymc3(posterior_predictive=post_pred), inplace=True)

In [ ]:
fig, ax = plt.subplots()
az.plot_ppc(idata, ax=ax)
ax.axvline(data.mean(), ls='--', color='r', label='True mean')
ax.legend(fontsize=10)

### Predicting on hold-out data

In many cases you want to predict on unseen / hold-out data. This is especially relevant in Probabilistic Machine Learning. We recently improved the API in this regard with the `pm.Data` container. It is a wrapper around a `theano.shared` variable whose values can be changed later. Otherwise, they can be passed into PyMC3 just like any other `numpy` array or tensor.

This distinction is significant since internally all models in PyMC3 are giant symbolic expressions. When you pass data directly into a model, you are giving Theano permission to treat this data as a constant and optimize it away as it sees fit. If you need to change this data later you might not have a way to point at it in the symbolic expression. Using `theano.shared` offers a way to point to a place in that symbolic expression, and change what is there.

In [ ]:
x = np.random.randn(100)
y = x > 0

with pm.Model() as model:
    # Create a shared variable that can be changed later on
    x_shared = pm.Data('x_obs', x)
    y_shared = pm.Data('y_obs', y)

    coeff = pm.Normal('x', mu=0, sigma=1)
    logistic = pm.math.sigmoid(coeff * x_shared)
    pm.Bernoulli('obs', p=logistic, observed=y_shared)
    idata = pm.sample(return_inferencedata=True)

Now assume we want to predict on unseen data. For this we have to change the values of `x_shared` and `y_shared`. Theoretically we don't neet to set `y_shared` as we want to predict it, but it has to match the shape of `x_shared`.

In [ ]:
with model:
    # Change the value and shape of the data
    pm.set_data(
        {
            'x_obs': [-1, 0, 1.0],
            # Use dummy values with the same shape:
            'y_obs': [0, 0, 0],
        }
    )

    post_pred = pm.sample_posterior_predictive(idata.posterior)

In [ ]:
post_pred['obs'].mean(axis=0)

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w